## Réseaux de neurones #3 : Classification de documents avec un réseau multicouches et des embeddings

On reprend dans cet exemple la classification de documents avec un réseau MLP à 2 couches.

Cependant, la principale différence est que la représentation d'un document est construite à partir des embeddings des mots de ce document. On utilise ici les embeddings de Spacy (note: les analyseurs de Spacy sont des réseaux de neurones).

### 1. Création du jeu de données

On utilise le même jeu de données que pour les exemples précédents.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

# On utilise le corpus 20Newsgroups et on limite les exemples d'entraînement à 4 classes
wanted_categories = ['rec.sport.hockey', 'sci.space', 'rec.autos', 'sci.med']

training_corpus = fetch_20newsgroups(subset='train', categories=wanted_categories, shuffle=True)
validation_corpus = fetch_20newsgroups(subset='test', categories=wanted_categories, shuffle=True)

target_categories = training_corpus.target_names

On crée également une instance de Spacy pour avoir accès à ses embeddings de mots.



In [2]:
import spacy 
nlp = spacy.load('en_core_web_lg')


## 2. Création d'une architecture neuronale multicouches

L'architecture est la même que pour l'exemple précédent, soit 2 couches linéaires avec une activation RELU sur la première couche.

On détermine également la taille des embeddings de Spacy (embedding_size). Cette valeur sera utilisée comme taille des vecteurs qui sont donnés en entrée au réseau (input_size).

In [3]:
from torch import nn

embedding_size = nlp.meta['vectors']['width'] # La dimension des vecteurs d'embeddings de Spacy
nb_classes = len(target_categories)

class MultiLayerPerceptron(nn.Module):
    
    def __init__(self, input_size, hidden_layer_size, output_size) :
        super().__init__()
        self.intput_layer = nn.Linear(input_size, hidden_layer_size)
        self.output_layer = nn.Linear(hidden_layer_size, output_size)
        
    def forward(self, x):
        x = self.intput_layer(x)
        x = nn.functional.relu_(x)
        x = self.output_layer(x)
        return x


## 3. Création d'un dataloader pour itérer dans les données en "minibatch"¶


In [4]:
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List
import numpy as np

class SpacyDataset(Dataset):
    
    def __init__(self, dataset: List[str] , target: np.array, sentence_aggregation_function):
        self.dataset = dataset
        self.aggregated_dataset = [None for _ in range(len(dataset))]
        self.sentence_aggregation_function = sentence_aggregation_function
        self.target = target
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.aggregated_dataset[index] is None:
            self.aggregated_dataset[index] = self.sentence_aggregation_function(self.dataset[index])  
        return FloatTensor(self.aggregated_dataset[index]), LongTensor([self.target[index]]).squeeze(0)

Dans cet exemple, le dataloader utilise une fonction qui permet de **fusionner les embeddings des mots d'un document en un seul vecteur qui représente le document**. Trois options sont disponibles:

* prendre la moyenne des embeddings (average_embedding)
* prendre la valeur maximum sur chacune des dimensions des embeddings (maxpool_embedding)
* utiliser une fonction de Spacy qui permet de représenter un texte par un seul embedding (spacy_cnn).

Pour fusionner les embeddings de mots ensemble (sauf spacy_cnn), la procédure est la suivante:

* tokéniser le texte en mots individuels
* aller chercher les embeddings de Spacy (token.vector) et les stocker dans une matrice (sentence_embedding_matrix)
* appliquer la fonction d'agrégation sur chacune des dimensions de la matrice pour obtenir un vecteur.

In [5]:
def average_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.meta['vectors']['width'] 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                  
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.average(sentence_embedding_matrix, axis=1)

def maxpool_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.meta['vectors']['width'] 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                    
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.max(sentence_embedding_matrix, axis=1)

def spacy_cnn(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    return tokenised_sentence.vector

Et on crée les dataloaders à l'aide de la fonction d'une fonction de fusion d'embeddings.


In [6]:
aggregation_function = average_embedding
# aggregation_function = maxpool_embedding
# aggregation_function = spacy_cnn

train_dataset = SpacyDataset(training_corpus.data, training_corpus.target, aggregation_function)
valid_dataset = SpacyDataset(validation_corpus.data, validation_corpus.target, aggregation_function)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)

## 4. Création d'une boucle d'entraînement
On peut maintenant entraîner le modèle comme on le faisait auparavant.

Note: Prévoir autre chose à faire pendant l'entraînement, car ça peut prendre un bon bout de temps :-)

In [7]:
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np

set_seeds(42)
hidden_size = 100

model = MultiLayerPerceptron(embedding_size, hidden_size, nb_classes)
experiment = Experiment('model/{}_mlp'.format(aggregation_function.__name__), 
                        model, 
                        optimizer = "SGD", 
                        task="classification")

In [8]:
logging = experiment.train(train_dataloader, valid_dataloader, epochs=100, disable_tensorboard=True)


Epoch: 1/100 Step: 149/149 100.00% |█████████████████████████|442.72s loss: 1.382166 acc: 28.013440 fscore_micro: 0.280134 val_loss: 1.379430 val_acc: 28.138801 val_fscore_micro: 0.281388
Epoch 1: val_acc improved from -inf to 28.13880, saving file to model/average_embedding_mlp\checkpoint_epoch_1.ckpt
Epoch: 2/100 Step: 149/149 100.00% |█████████████████████████|1.39s loss: 1.376620 acc: 36.455271 fscore_micro: 0.364553 val_loss: 1.374231 val_acc: 42.649842 val_fscore_micro: 0.426498
Epoch 2: val_acc improved from 28.13880 to 42.64984, saving file to model/average_embedding_mlp\checkpoint_epoch_2.ckpt
Epoch: 3/100 Step: 149/149 100.00% |█████████████████████████|1.44s loss: 1.371278 acc: 51.910962 fscore_micro: 0.519110 val_loss: 1.369187 val_acc: 47.003155 val_fscore_micro: 0.470032
Epoch 3: val_acc improved from 42.64984 to 47.00315, saving file to model/average_embedding_mlp\checkpoint_epoch_3.ckpt
Epoch: 4/100 Step: 149/149 100.00% |█████████████████████████|1.41s loss: 1.365761 a

Epoch 23: val_acc improved from 80.12618 to 82.46057, saving file to model/average_embedding_mlp\checkpoint_epoch_23.ckpt
Epoch: 24/100 Step: 149/149 100.00% |█████████████████████████|1.46s loss: 0.890585 acc: 84.712306 fscore_micro: 0.847123 val_loss: 0.878513 val_acc: 82.586751 val_fscore_micro: 0.825868
Epoch 24: val_acc improved from 82.46057 to 82.58675, saving file to model/average_embedding_mlp\checkpoint_epoch_24.ckpt
Epoch: 25/100 Step: 149/149 100.00% |█████████████████████████|1.43s loss: 0.855073 acc: 85.888282 fscore_micro: 0.858883 val_loss: 0.844316 val_acc: 82.397476 val_fscore_micro: 0.823975
Epoch: 26/100 Step: 149/149 100.00% |█████████████████████████|1.47s loss: 0.820291 acc: 86.770265 fscore_micro: 0.867703 val_loss: 0.811806 val_acc: 83.848580 val_fscore_micro: 0.838486
Epoch 26: val_acc improved from 82.58675 to 83.84858, saving file to model/average_embedding_mlp\checkpoint_epoch_26.ckpt
Epoch: 27/100 Step: 149/149 100.00% |█████████████████████████|1.43s loss

Epoch: 46/100 Step: 149/149 100.00% |█████████████████████████|1.50s loss: 0.390586 acc: 92.986140 fscore_micro: 0.929861 val_loss: 0.394030 val_acc: 91.419558 val_fscore_micro: 0.914196
Epoch 46: val_acc improved from 91.16719 to 91.41956, saving file to model/average_embedding_mlp\checkpoint_epoch_46.ckpt
Epoch: 47/100 Step: 149/149 100.00% |█████████████████████████|1.62s loss: 0.380407 acc: 93.070139 fscore_micro: 0.930701 val_loss: 0.385530 val_acc: 91.356467 val_fscore_micro: 0.913565
Epoch: 48/100 Step: 149/149 100.00% |█████████████████████████|1.50s loss: 0.370162 acc: 93.196136 fscore_micro: 0.931961 val_loss: 0.376183 val_acc: 91.545741 val_fscore_micro: 0.915457
Epoch 48: val_acc improved from 91.41956 to 91.54574, saving file to model/average_embedding_mlp\checkpoint_epoch_48.ckpt
Epoch: 49/100 Step: 149/149 100.00% |█████████████████████████|1.42s loss: 0.360395 acc: 93.364133 fscore_micro: 0.933641 val_loss: 0.368694 val_acc: 91.230284 val_fscore_micro: 0.912303
Epoch: 5

Epoch: 71/100 Step: 149/149 100.00% |█████████████████████████|2.13s loss: 0.234763 acc: 95.044099 fscore_micro: 0.950441 val_loss: 0.253342 val_acc: 93.123028 val_fscore_micro: 0.931230
Epoch: 72/100 Step: 149/149 100.00% |█████████████████████████|2.28s loss: 0.231641 acc: 94.918102 fscore_micro: 0.949181 val_loss: 0.249828 val_acc: 93.123028 val_fscore_micro: 0.931230
Epoch: 73/100 Step: 149/149 100.00% |█████████████████████████|1.93s loss: 0.227494 acc: 94.918102 fscore_micro: 0.949181 val_loss: 0.246394 val_acc: 93.186120 val_fscore_micro: 0.931861
Epoch: 74/100 Step: 149/149 100.00% |█████████████████████████|1.72s loss: 0.224682 acc: 95.254095 fscore_micro: 0.952541 val_loss: 0.248061 val_acc: 92.996845 val_fscore_micro: 0.929968
Epoch: 75/100 Step: 149/149 100.00% |█████████████████████████|1.57s loss: 0.221459 acc: 95.254095 fscore_micro: 0.952541 val_loss: 0.240817 val_acc: 93.375394 val_fscore_micro: 0.933754
Epoch 75: val_acc improved from 93.31230 to 93.37539, saving file

Epoch: 96/100 Step: 149/149 100.00% |█████████████████████████|1.63s loss: 0.173216 acc: 95.842083 fscore_micro: 0.958421 val_loss: 0.205705 val_acc: 93.438486 val_fscore_micro: 0.934385
Epoch: 97/100 Step: 149/149 100.00% |█████████████████████████|1.56s loss: 0.171738 acc: 96.052079 fscore_micro: 0.960521 val_loss: 0.202211 val_acc: 93.690852 val_fscore_micro: 0.936909
Epoch: 98/100 Step: 149/149 100.00% |█████████████████████████|1.57s loss: 0.169720 acc: 96.094078 fscore_micro: 0.960941 val_loss: 0.200957 val_acc: 94.195584 val_fscore_micro: 0.941956
Epoch: 99/100 Step: 149/149 100.00% |█████████████████████████|1.48s loss: 0.167774 acc: 96.178076 fscore_micro: 0.961781 val_loss: 0.198187 val_acc: 93.817035 val_fscore_micro: 0.938170
Epoch: 100/100 Step: 149/149 100.00% |█████████████████████████|1.53s loss: 0.166191 acc: 95.926081 fscore_micro: 0.959261 val_loss: 0.196548 val_acc: 94.321767 val_fscore_micro: 0.943218
Epoch 100: val_acc improved from 94.25868 to 94.32177, saving fi

## 5. Prédiction sur quelques exemples


In [9]:
from torch.nn.functional import softmax

def get_most_probable_class(sentence, model):
    vectorized_sentence = aggregation_function(sentence)
    prediction = model(FloatTensor(vectorized_sentence).squeeze(0)).detach()
    output = softmax(prediction, dim=0)
    max_category_index = np.argmax(output)
    max_category = target_categories[max_category_index]
    print("\nClassification de la phrase: ", sentence)
    print("Sorties du réseau de neurones:", prediction)
    print("Valeurs obtenues après application de softmax:", output)
    print("Meilleure classe: {} qui correspond en sortie au neurone {}".format(max_category, max_category_index))
    return(max_category)


In [10]:
# On test le modèle avec de nouvelles phrases 

test_docs = ['Getzky was a center, not a goaltender', 
             'Mazda and BMW cars are esthetic',
             'Doctor, doctor, gimme the news', 
             'Take me to the moon']

[get_most_probable_class(sentence, model) for sentence in test_docs]



Classification de la phrase:  Getzky was a center, not a goaltender
Sorties du réseau de neurones: tensor([-4.0090,  7.3172, -2.5695, -0.4598])
Valeurs obtenues après application de softmax: tensor([1.2048e-05, 9.9952e-01, 5.0825e-05, 4.1907e-04])
Meilleure classe: rec.sport.hockey qui correspond en sortie au neurone 1

Classification de la phrase:  Mazda and BMW cars are esthetic
Sorties du réseau de neurones: tensor([ 23.5277,  -3.9371, -10.8859,  -8.7623])
Valeurs obtenues après application de softmax: tensor([1.0000e+00, 1.1808e-12, 1.1334e-15, 9.4759e-15])
Meilleure classe: rec.autos qui correspond en sortie au neurone 0

Classification de la phrase:  Doctor, doctor, gimme the news
Sorties du réseau de neurones: tensor([-4.9605, -3.8526, 12.3112, -3.3667])
Valeurs obtenues après application de softmax: tensor([3.1550e-08, 9.5528e-08, 1.0000e+00, 1.5529e-07])
Meilleure classe: sci.med qui correspond en sortie au neurone 2

Classification de la phrase:  Take me to the moon
Sorties 

['rec.sport.hockey', 'rec.autos', 'sci.med', 'sci.space']